In [3]:
import torch 
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:
torch.random.manual_seed(0) 

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model.eval()

Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.18s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [6]:
prompts = {'landlord_prompt' : "You are a real estate expert giving advice to a landlord in a Jeonse (Korean key money deposit system) transaction. Provide some useful tips to ensure a smooth and successful rental agreement. Focus on financial security, contract terms, and tenant relations.",
'tenant_prompt' : "You are a real estate expert giving advice to a tenant in a Jeonse (Korean key money deposit system) transaction. Provide some useful tips to ensure the tenant's rights and security during the rental period. Focus on deposit safety, contract understanding, and maintenance."}

In [8]:
messages = [ 
    {"role": "system", "content": prompts['landlord_prompt']}, 
    {"role": "user", "content": "집주인은 소유권을 가지고 있는 것이잖아요, 그러면 집주인은 항상 임대인이 되는 것인가요?"}, 
    {"role": "assistant", "content": "아닙니다. 집주인은 건물 소유권을 가졌을 뿐이지 집주인은 임대인은 아닙니다. 집주인이 아닌 사람이 임대 권한만 부여받을 수도 있습니다. 예를들면 부부 공동 명의로 아파트를 가져도 남편만 임대인이 될 수 있습니다. 이 때 아내는 남편에게 임대 권한을 위임한 것입니다. "}, 
    {"role": "user", "content": "전세거래 전 임대인에게 몇가지 조언을 해줄래?"}, 
] 

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [14]:
outputs = model.generate(
    input_ids,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


물론입니다. 전세거래가 한국의 주거시장에서 중요한 거래 형태 중 하나입니다. 임대인으로서 성공적인 전세 계약을 위해 고려해야 할 몇 가지 사항을 소개합니다.

### 1. **적절한 임대료 설정**
- **시장 조사**: 임대료를 결정할 때는 해당 지역의 시장 조사를 철저히 해야 합니다. 유사한 아파트나 건물의 임대료를 조사하여 적절한 임대료를 설정하세요.
- **법적 기준 준수**: 임대료는 법적 기준을 초과하지 않도록 주의해야 합니다. 이는 임대료 인상 시에도 마찬가지입니다.

### 2. **전세금 관리**
- **예산 수립**: 전세보증금과 임대료를 받을 때부터 예산을 세워야 합니다. 예산이 부족하면 임대인에게 부담이 될 수 있습니다.
- **보전 관리**: 보증금은 법적으로 보호받는 돈이므로, 임대인이 법적으로 요구할 수 있는 경우에만 사용하세요.

### 3. **계약서 작성**
- **법적 조항 포함**: 계약서에 법적 조항을 포함하여, 예를 들어, 계약 해지 조건, 임대료 인상 조건, 보증금 반환 조건 등을 명확히 명시하세요.
- **상호 이해**: 계약서 작성 시 임차인과 상호 이해를 충분히 하고, 계약서에 모든 사항을 명확히 기재하세요.

### 4. **임차인과의 관계 관리**
- **소통 중요**: 임차인과의 소통은 매우 중요합니다. 문제 발생 시 신속하게 대응하고, 임차인의 불만을 사전에 해결하려 노력하세요.
- **법적 절차 준수**: 법적 문제가 발생했을 경우, 법적 절차를 준수하여 해결하세요. 임차인과의 갈등은 법적 문제로 이어질 수 있으므로, 법률 전문가의 도움을 받는 것이 좋습니다.

### 5. **상태 점검**
- **정기 점검**: 정기적으로 임대 부동산의 상태를 점검하여, 임차인으로부터의 손상이나 파손이 없음을 확인하세요.
- **보수 책임**: 임차인으로부터 발생하는 손상이나 파손에 대한 보수 책임을 명확히 하여, 임대인에게 부담이 되지 않도록 하세요.

### 6. **보험 가입**
- **임대주택 보험**: 임대주택